# <center> MATH&ML-6. Математический анализ в контексте задачи оптимизации. Часть III

In [2]:
#Импорты
import numpy as np
import pandas as pd
import sympy
from sympy import Eq, solveset, Symbol, symbols, Interval, S, log, sin, cos, exp, diff, solve, N
# Аккуратный вывод формул
from sympy import init_printing

from scipy import optimize
from scipy.optimize import minimize

from sympy.calculus.util import function_range, continuous_domain

### Юнит 1. Введение

In [3]:
# Задание 1.1

x, w = symbols('x w')
func = -x**4 + 6*x**3 - 4*x**2 + 80
display(func.diff(x))
display(solve(func.diff(x)))

print('Ответ:',-4**4 + 6*4**3 - 4*4**2 + 80)

-4*x**3 + 18*x**2 - 8*x

[0, 1/2, 4]

Ответ: 144


In [4]:
#Задание 1.4
x, y, w = symbols('x y w')
func = x**2 + 2*y**2
g = x + y - 20

full_func = func + w*g
display(f'diff x: {full_func.diff(x)}')
display(f'diff y: {full_func.diff(y)}')
display(f'diff w: {full_func.diff(w)}')
xyw = solve([full_func.diff(x), full_func.diff(y), full_func.diff(w)], x, y, w)
display(xyw)
display(f'Ответ: {(xyw[x]**2 + 2*xyw[y]**2)*3}')

'diff x: w + 2*x'

'diff y: w + 4*y'

'diff w: x + y - 20'

{w: -80/3, x: 40/3, y: 20/3}

'Ответ: 800'

### Юнит 2. Градиентный спуск: применение и модификации

In [5]:
# Задание 2.7
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn import metrics
df = sns.load_dataset('diamonds')
# display(df)

df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
df = pd.get_dummies(df, drop_first=True)
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

X = df.drop(columns="price")
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

param_grid = [
    {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 10),
    "l1_ratio": np.linspace(0, 1, 10),
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
    }
]

grid_search = GridSearchCV(
    estimator=SGDRegressor(
        random_state=42,
        max_iter=1200
    ),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

%time grid_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.3f}".format(grid_search.score(X_test, y_test)))
y_test_pred = grid_search.predict(X_test)
print('MSE на тестовом наборе: {:.3f}'.format(metrics.mean_squared_error(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 26.8 s
Wall time: 37.6 s
accuracy на тестовом наборе: 0.957
MSE на тестовом наборе: 0.044
Наилучшие значения гиперпараметров: {'alpha': np.float64(0.001), 'eta0': np.float64(0.001), 'l1_ratio': np.float64(0.0), 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}


### Юнит 3. Метод Ньютона

In [6]:
# Задание 3.1

x = symbols('x')
func = 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
display(func.diff())
x0 = 0.7
x1 = N(x0 - (func.subs(x, x0).evalf())/(func.diff().subs(x, x0)), 3)
x2 = N(x1 - (func.subs(x, x1).evalf())/(func.diff().subs(x, x1)), 3) # Нужный
x3 = N(x2 - (func.subs(x, x2).evalf())/(func.diff().subs(x, x2)), 3)

print(x0, x1, x2, x3)

#Автоматическое решение Deepseek (По сути реализация градиентного спуска, для этого задания не подходит, он не понял, что от него хотят)
learning_rate = 0.1
current_x = 0.7  # Начальная точка
n_iterations = 5

for i in range(n_iterations):
    grad = func.subs(x, current_x).evalf()  # Вычисляем градиент в текущей точке
    current_x = current_x - learning_rate * grad  # Обновляем x (градиентный спуск)
    current_f = func.subs(x, current_x).evalf()      # Значение функции в новой точке
    print(f"Iter {i+1}: x = {current_x:.4f}, f(x) = {current_f:.4f}, grad = {grad:.4f}")

30*x**4 - 20*x**3 - 12*x**2 + 6*x

0.7 0.630 0.629 0.628
Iter 1: x = 0.7094, f(x) = -0.1066, grad = -0.0941
Iter 2: x = 0.7201, f(x) = -0.1206, grad = -0.1066
Iter 3: x = 0.7321, f(x) = -0.1361, grad = -0.1206
Iter 4: x = 0.7457, f(x) = -0.1531, grad = -0.1361
Iter 5: x = 0.7611, f(x) = -0.1711, grad = -0.1531


In [7]:
func0 = x**3 - 3*x**2 - 45*x - 40

def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

#Ручной цикл
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
    
# Через функцию
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

display(newtons_method(f=func1, fprime=func2, x0=42, tol=0.0001))

from scipy.optimize import newton
display(newton(func=func1, fprime=func2, x0=42, tol=0.0001))

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


5.000000000000001

np.float64(5.000000000105126)

In [8]:
# Задание 3.6
func = x**3 - 72*x - 220
x0 = 12

x = symbols('x')
x1 = x0 - (func.subs(x, x0)/func.diff().subs(x, x0))
x2 = x1 - (func.subs(x, x1)/func.diff().subs(x, x1))
x3 = x2 - (func.subs(x, x2)/func.diff().subs(x, x2))
x4 = x3 - (func.subs(x, x3)/func.diff().subs(x, x3))

print(x0, x1, N(x2, 4), N(x3, 4), N(x4, 4))

#Или

def func(x):
    return x**3 - 72*x - 220
np.round(newton(func=func, x0=x0), 3)

12 919/90 9.756 9.727 9.727


np.float64(9.727)

In [9]:
# Задание 3.7
func = x**2 + 9*x - 5
x0 = 2.2

x = symbols('x')
x1 = x0 - (func.subs(x, x0)/func.diff().subs(x, x0))
x2 = x1 - (func.subs(x, x1)/func.diff().subs(x, x1))
x3 = x2 - (func.subs(x, x2)/func.diff().subs(x, x2))
x4 = x3 - (func.subs(x, x3)/func.diff().subs(x, x3))
x5 = x4 - (func.subs(x, x4)/func.diff().subs(x, x4))
print(x0, x1, N(x2, 3), N(x3, 3), N(x4, 3), N(x5, 3)) #Ответ почему-то 0.52, 
#хотя при округлении должно быть 0.53, решение соответствует эталону

# или
def func(x):
    return x**3 - 72*x - 220
np.round(newton(func=func, x0=x0, tol=0.01), 2)

2.2 0.734328358208955 0.529 0.525 0.525 0.525


np.float64(-3.85)

In [10]:
#Задание 3.9
def func(x):
    return 8*x**3 - 2*x**2 - 450
def funcdiff1(x):
    return 24*x**2 - 4*x
def funcdiff2(x):
    return 48*x

x0 = 42

def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr
display(round(newtons_method(f=funcdiff1, fprime=funcdiff2, x0=x0, tol=0.0001), 3))

np.round(newton(func=funcdiff1, fprime=funcdiff2, x0=x0, tol=0.0001), 3)

0.167

np.float64(0.167)

### Юнит 4. Квазиньютоновские методы

In [11]:
# определение функции
def func(x):
    return x[0]**2.0 + x[1]**2.0

# Определение градиента
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

x_0 = [1.0, 1.0]

result = minimize(func, x_0, method='BFGS', jac=grad_func)

print('Статус оптимизации: %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
print('-'*20)

# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации: Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000
--------------------
Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [20]:
# Задание 4.1
def func(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

x_0 = [-400, -400]

print('BFGS-решение:')
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print(result)
print('-'*20)

print('L-BFGS-B решение:')
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
print(result)
print('-'*20)

BFGS-решение:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.0
        x: [-4.000e+00  1.000e+00]
      nit: 6
      jac: [-1.776e-15  0.000e+00]
 hess_inv: [[ 6.667e-01  3.333e-01]
            [ 3.333e-01  6.667e-01]]
     nfev: 11
     njev: 11
--------------------
L-BFGS-B решение:
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -0.9999999999999218
        x: [-4.000e+00  1.000e+00]
      nit: 4
      jac: [ 2.900e-07  2.724e-07]
     nfev: 9
     njev: 9
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
--------------------


In [18]:
print('Эталон:')
def func(x):
    return x[0] ** 2 - x[0] * x[1] + x[1] ** 2 + 9 * x[0] - 6 * x[1] + 20

def grad_func(x):
    return np.array([2 * x[0] - x[1] + 9, -x[0] + 2 * x[1] - 6])

x_0 = [-400, -400]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print(result)

Эталон:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.0
        x: [-4.000e+00  1.000e+00]
      nit: 6
      jac: [-1.776e-15  0.000e+00]
 hess_inv: [[ 6.667e-01  3.333e-01]
            [ 3.333e-01  6.667e-01]]
     nfev: 11
     njev: 11


In [ ]:
# Задание 4.4
def func(x):
    return x**2 - 3*x + 45

def grad_func(x):
    return 2*x - 3

x_0 = 10
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print(result)
print('-'*20)

#Задание 4.5
def func(x):
    return x**2 - 3*x + 45

def grad_func(x):
    return 2*x - 3

x_0 = 10
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
print(result)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 4
      jac: [ 0.000e+00]
 hess_inv: [[ 5.000e-01]]
     nfev: 5
     njev: 5
--------------------
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 2
      jac: [ 0.000e+00]
     nfev: 3
     njev: 3
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>


In [24]:
#Задание 4.7
def func(x):
    return x[0]**4 + 6*x[1]**2 + 10

def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])

x_0 = [100, 100]
result = minimize(func, x_0, method='BFGS', jac=grad_func)
print('BFGS:')
print(result)
print('-'*20)

result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
print('L-BFGS-B:')
print(result)
print('-'*20)

BFGS:
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 10.000000030008898
        x: [ 1.316e-02  6.653e-14]
      nit: 34
      jac: [ 9.120e-06  7.984e-13]
 hess_inv: [[ 2.016e+02 -4.163e-09]
            [-4.163e-09  7.317e-02]]
     nfev: 37
     njev: 37
--------------------
L-BFGS-B:
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 10.00000000827103
        x: [-9.527e-03 -2.322e-06]
      nit: 37
      jac: [-3.459e-06 -2.786e-05]
     nfev: 40
     njev: 40
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
--------------------
